### Example usage for calculate_metrics

In [27]:
import pickle
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils.audio_to_spectrograms import create_spectrogram_pkl
#from src.main import calculate_metrics

# Copied from src.main as I'm having problems with audio mamba imports
from src.config import CLASSES
from loguru import logger
import sed_eval

def calculate_metrics(pred_event_dict, gt_event_dict, time_resolution=1.0, t_collar=0.25):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES, t_collar=t_collar)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        event_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(event_based_metrics)

    segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(CLASSES, time_resolution=time_resolution)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        segment_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(segment_based_metrics)

In [19]:
# refer to single_stage_yamnet_frame.ipynb (Section "Get test set event predictions") for pickle file generation
pred_event_dict = pickle.load(open('outputs/single_stage_yamnet.pkl', 'rb'))

In [20]:
pred_event_dict['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': 0.975609756097561,
  'event_offset': 3.902439024390244,
  'event_label': 'siren'},
 {'file': 'test_snipped_scene_0000.wav',
  'event_onset': 3.902439024390244,
  'event_offset': 5.365853658536585,
  'event_label': 'car_horn'},
 {'file': 'test_snipped_scene_0000.wav',
  'event_onset': 5.365853658536585,
  'event_offset': 5.853658536585366,
  'event_label': 'gun_shot'}]

In [21]:
# check if pkl file is created
gt_pkl_path = 'data/processed/yamnet/spectrograms_test_list.pkl'
if not(os.path.exists(gt_pkl_path)):
    create_spectrogram_pkl()

In [22]:
gt_events = pickle.load(open(gt_pkl_path, 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [23]:
gt_event_dict['test_snipped_scene_0000.wav']

[{'file': 'test_snipped_scene_0000.wav',
  'event_onset': np.float64(1.5601864044243652),
  'event_offset': np.float64(5.414698876079694),
  'event_label': 'siren'}]

In [28]:
calculate_metrics(pred_event_dict, gt_event_dict, t_collar=0.5)

2025-12-02 00:50:00.598 | WARNING  | __main__:calculate_metrics:15 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 4111.27 sec
  Evaluated files                   : 500 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 500.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 38.21 %
    Precision                       : 30.77 %
    Recall                          : 50.40 %
  Error rate
    Error rate (ER)                 : 1.61 
    Substitution rate               : 0.02 
    Deletion rate                   : 0.47 
    Insertion rate                  : 1.11 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 39.62 %
    Precision                       : 32.61 %
    Recall                          : 50.71 %
  Error rate
    Error rate (ER)                 : 1.62 
    Deletion rate                   : 0.49 
    Insertion rate 